# Interface for Pipeline Execution

## 1) Import Models, Metrics, Paths, and Functions

In [1]:
from utils.helpers import csv_reader
from utils.paths import *
from pipeline.run_pipeline import run_pipeline


from models import MODELS
from metrics import METRICS


Loading paths...
Loading helper functions...
Loading data transformers...
Loading models...
Loading metrics...


In [2]:
# print(METRICS)
# print(MODELS)

## 2) Select Input Data 

In [3]:
# Read input data
df = csv_reader(SIMDATA_DIR, 'noisy_simdata')
print(df.head())

# For testng pipeline for now I use a subset of 10 years:
df = df.iloc[:120, :]

                    y          x1         x2          x3
Date                                                    
2004-01-01  50.840469  427.595799  55.337904  900.325291
2004-02-01  52.871538  434.062163  54.959155  900.775888
2004-03-01  53.769316  453.264284  56.470633  899.510058
2004-04-01  57.672973  459.367523  56.704233  903.524834
2004-05-01  57.182051  462.354356  61.557907  905.071762


In [4]:
# ## Using the EUR-USD Exchange Rate data
# 
# df = csv_reader(TESTDATA_DIR, 'eurusd', columns=['datetime', 'bid_close'])
# import sys
# import warnings
# # warnings( sys.__stdout__())
# 
# #sys.jupyter_stdout = sys.__stdout__
# print("hey")
# print(df.head())
# 
# # For testng pipeline for now I use a small subset:
# df = df.iloc[:10000, :]

## 3) Run Pipeline

### 3.1) ... on Simulated Data

In [5]:
output_dict = run_pipeline(
    df=df, models=MODELS, metrics=METRICS,
    select_forecasters=['all'],
    verbose=True
)

[2024-03-06 10:18] Starting  Pipeline...

== Starting Step 1 in Pipeline: Data Preprocessing ==

Searching for time information...
Dates found in 'index' column!
Inferred frequency: month start
Data from goes from 2004-01-01 to 2013-12-01, resulting in 120 observations.

Selecting target and covariates...
Target: y
Covariates: x1, x2, x3

Data Insight:

                    y          x1         x2          x3
Date                                                    
2004-01-01  50.840469  427.595799  55.337904  900.325291
2004-02-01  52.871538  434.062163  54.959155  900.775888
2004-03-01  53.769316  453.264284  56.470633  899.510058
2004-04-01  57.672973  459.367523  56.704233  903.524834
2004-05-01  57.182051  462.354356  61.557907  905.071762
[Time elapsed: 00s]


== Starting Step 2 in Pipeline: Individual Forecasts ==

Splitting data (train/test ratio: 30/70)...
Initial training set has 36 observations 
and goes from 2004-01-01 to 2006-12-01
There are 84 periods to be forecasted: 
2

### 3.2) ... on EUR-USD Exchange Data

In [6]:
# output_dict = (
#     run_pipeline(
#         df=df, models=MODELS, metrics=METRICS,
#         agg_method='last', agg_freq='B',
#         forecast_init_train=0.3, ensemble_init_train=0.3,
#         autosarimax_refit_interval=0.33,
#         select_forecasters=['Naive', 'STL'],
#         select_ensemblers=['Simple', 'Inverse RMSE'],
#         verbose=True
#     )
# )

## 4) Show Ranking Table

In [7]:
import pandas as pd
pd.set_option('display.float_format', '{:.2f}'.format)
# Format performance metric values
def format_numeric(val):
    if isinstance(val, float):
        return '{:.3f}'.format(val)
    return val
formatted_metrics = output_dict['metrics ranking'].applymap(format_numeric)
display(formatted_metrics.style.hide())

Model,MAE,RMSE,MAPE,sMAPE,MAE Ranking,RMSE Ranking,MAPE Ranking,sMAPE Ranking
Weighted Ensemble: Inverse RMSE,1.985,2.615,0.028,1.845,1,1,1,1
Weighted Ensemble: Simple,1.999,2.641,0.028,1.857,2,2,2,2
Weighted Ensemble: Inverse Variance,2.006,2.643,0.028,1.865,3,3,3,3
Exponential Smoothing,2.073,2.707,0.029,1.923,4,4,4,4
Naive,2.090,2.748,0.029,1.941,5,6,5,5
AutoSARIMA,2.111,2.731,0.029,1.954,6,5,6,6
Naive (drift),2.118,2.748,0.029,1.964,7,7,7,7
Meta Ensemble: RandomForest,2.181,2.782,0.030,2.024,8,8,8,8
Meta Ensemble: Ridge,2.203,2.888,0.031,2.046,9,9,9,9
Weighted Ensemble: Inverse Error Covariance,2.244,2.998,0.031,2.089,11,12,10,10
